In [63]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd
import numpy as np
es = Elasticsearch(http_compress=True, timeout=1000)

In [3]:
#clean data (fix blank values)


In [4]:
t_df = pd.read_json(".\\tropical_job.2020-11-05\\topics_combined_processed.json", lines=True,convert_dates=False)

In [9]:
dfs = [t_df]

In [10]:
t_df.rename({'parentLink':'threadID'},axis='columns',inplace=True)

In [11]:
#extract data from arrays, i.e. removing the brackets around the array

for column in ['username','dateTime','postID','mainText','threadID']:
    for df in dfs:
        df[column] = df[column].str[0]

In [12]:
t_df['threadID'] = t_df['threadID'].str.split('=').str[-1]

In [13]:
t_df['dateTime_raw'] = pd.to_datetime(t_df['dateTime'],format=' %B %d, %Y, %I:%M:%S %p »',errors='coerce')

In [ ]:
t_df.isnull().sum()

In [14]:
for df in dfs:
    df.drop(['timestamp','dateTime'],axis='columns',inplace=True)

In [18]:
t_df.drop(['mainTextLinks','quoteLink'],axis='columns',inplace=True)

In [50]:
t_df['mainText'].fillna('',inplace=True)
t_df['username'].fillna('',inplace=True)
t_df['dateTime_raw'].fillna(pd.Timestamp.min.ceil('S'),inplace=True)

In [ ]:
t_df[t_df['dateTime_raw'].isna()]

In [34]:
test_df = pd.read_json(".\\tropical_job.2020-11-05\\topics_combined_processed.json", lines=True,convert_dates=False)

In [58]:
test_df.iloc[2]

username                                        [pj1881 (Patrick)]
timestamp                                              1.60472e+09
dateTime                        [ January 14, 2012, 07:23:02 PM »]
mainTextLinks                                                 None
quoteLink                                                     None
mainText                                                   [Salak]
postID                                                        [38]
parentLink       [http://tropicalfruitforum.com/index.php?PHPSE...
Name: 2, dtype: object

In [55]:
df_iter_test = df.iterrows()

In [60]:
#generate doc per row
def doc_yield(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield { "_index": index,
                "_type": "_doc",
                "_id": document['postID'],
                "_source": document.to_dict()
              }
    raise StopIteration

In [64]:
helpers.bulk(es, doc_yield(t_df))

C:\Users\PC\anaconda3\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


BulkIndexError: ('500 document(s) failed to index.', [{'index': {'_index': '500', '_type': '_doc', '_id': '62805', 'status': 400, 'error': {'type': 'validation_exception', 'reason': 'Validation Failed: 1: this action would add [2] total shards, but this cluster currently has [1000]/[1000] maximum shards open;'}, 'data': {'username': 'jc', 'mainText': 'Thank you fine Sir.', 'postID': '62805', 'threadID': '1001.0', 'dateTime_raw': Timestamp('2013-03-05 21:57:41')}}}, {'index': {'_index': '501', '_type': '_doc', '_id': '62807', 'status': 400, 'error': {'type': 'validation_exception', 'reason': 'Validation Failed: 1: this action would add [2] total shards, but this cluster currently has [1000]/[1000] maximum shards open;'}, 'data': {'username': 'DuncanYoung', 'mainText': 'Bonide Iron Drench, has all your minors, applied as a soil drench, along with osmocote granules should perk it right up.', 'postID': '62807', 'threadID': '1001.0', 'dateTime_raw': Timestamp('2013-03-05 22:17:33')}}}, {'in

In [66]:
res = elastic_search.get(index="test-index", id=1)

NameError: name 'elastic_search' is not defined